# SWB Mood GLMs
Updated: 11/20/2024

In [12]:
import numpy as np
import pandas as pd
from glob import glob
from numpy.linalg import vecdot
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr, norm 
import scipy.stats as stats
from scipy.optimize import least_squares, minimize
from sklearn.metrics import r2_score
import random
import os
# from statannot import add_stat_annotation
from statsmodels.stats.outliers_influence import variance_inflation_factor
import itertools
import datetime
import statsmodels.api as sm
from joblib import Parallel, delayed
from tqdm import tqdm
import pickle





In [13]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [26]:
base_dir  = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/'
paper_dir = f'{base_dir}manuscript/'
behav_dir = f'{paper_dir}data/behav/'
pt_dir    = f'{paper_dir}results/behav/subj_pt_data/'
fig_dir   = f'{paper_dir}figs/behav/'
os.makedirs(fig_dir,exist_ok=True)

date = datetime.date.today().strftime('%m%d%Y')
print(date)


11212024


In [27]:
# import sys
# sys.path.append(f'{base_dir}swb_computational_modeling/swb_behav_models/scripts')
# from SWB_modeling_utils import *

import sys
sys.path.append(f'{paper_dir}scripts/behav/')
from behav_utils import *
from swb_subj_behav import *
from SWB_modeling_utils import *
# sys.path.append(f'{paper_dir}scripts/behav/pyEM-main/')
# from pyEM import fitting
# from pyEM import plotting
# from pyEM import math

In [28]:
date = datetime.date.today().strftime('%m%d%Y')
print(date)

11212024


In [29]:
##### MS050 + MS028 CANNOT BE INCLUDED - THEY DID NOT DO THE MOOD TASK

subj_ids = pd.read_excel(f'{base_dir}SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID.tolist()
subj_ids.remove('MS050')
subj_ids.remove('MS028')
n_subj = len(subj_ids)
subj_ids



['MS002',
 'MS003',
 'MS009',
 'MS011',
 'MS015',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS024',
 'MS025',
 'MS026',
 'MS027',
 'MS029',
 'MS030',
 'MS033',
 'MS035',
 'MS041',
 'MS043',
 'MS048',
 'DA8',
 'DA023',
 'DA026',
 'DA037',
 'DA039']

# Load behav dfs

In [30]:
# load task_dfs into master list 
raw_behav = [pd.read_csv(f'{behav_dir}single_subj/{subj_id}_task_df.csv') for subj_id in subj_ids]
all_behav,beh_drops = format_all_behav(raw_behav, return_drops=True, drop_bads=False, drop_bads_t1=False,
                     norm=True,norm_type='zscore',pt_dir=pt_dir)
all_behav

,subj_id,bdi,bdi_thresh,Round,TrialNum,RT,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,...,P_Gamble_raw,P_Gamble_t1_raw,risk_raw,risk_t1_raw,P_Safe_raw,P_Safe_t1_raw,util_Gamble_raw,util_Gamble_t1_raw,util_HighBet_raw,util_HighBet_t1_raw
0,MS002,14,low,1,25.0,2.059852,513.380590,513.390239,515.450091,515.457173,...,0.101995,0.361720,0.819074,0.819074,0.898005,0.638280,-0.573858,-0.771765,0.031011,-0.995920
1,MS002,14,low,2,117.0,1.954564,522.640856,522.641563,524.596127,526.627092,...,0.361720,0.658866,0.819074,0.819074,0.638280,0.341134,-0.769643,1.408898,-0.999461,1.493433
2,MS002,14,low,3,79.0,1.583462,531.174799,531.175599,532.759061,534.780269,...,0.658866,0.285628,0.819074,0.819074,0.341134,0.714372,1.415906,0.064982,1.498223,0.893798
3,MS002,14,low,4,42.0,2.491611,545.592613,545.593355,548.084966,548.092333,...,0.285628,0.516600,0.819074,0.819074,0.714372,0.483400,0.068979,1.215829,0.896581,0.945238
4,MS002,14,low,5,85.0,1.768936,555.337336,555.345720,557.114656,559.135069,...,0.516600,0.117552,0.819074,0.819074,0.483400,0.882448,1.222404,-0.495163,0.948193,0.261826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,DA039,22,high,146,79.0,1.079701,2259.827656,2259.828749,2260.908450,2262.926195,...,0.790182,0.276652,1.186903,1.186903,0.209818,0.723348,1.553154,-0.783149,1.505115,-0.112887
3746,DA039,22,high,147,30.0,1.837272,2267.502359,2267.534059,2269.371331,2269.377701,...,0.276652,0.490831,1.186903,1.186903,0.723348,0.509169,-0.788844,-0.065859,-0.133157,-0.373865
3747,DA039,22,high,148,13.0,4.030006,2282.349445,2282.350662,2286.380667,2286.389886,...,0.490831,0.414647,1.186903,1.186903,0.509169,0.585353,-0.078197,-0.304923,-0.381965,-0.360612
3748,DA039,22,high,149,18.0,3.167144,2293.040983,2293.042042,2296.209186,2296.218136,...,0.414647,0.115720,1.186903,1.186903,0.585353,0.884280,-0.315047,-2.279002,-0.369331,-0.830635


In [31]:
raw_mood = [pd.read_csv(f'{behav_dir}single_subj/{subj_id}_mood_df.csv') for subj_id in subj_ids]
all_mood, mood_drops = format_all_mood(raw_mood, all_behav, return_drops=True, drop_bads=False)


In [32]:
all_mood

,subj_id,bdi,bdi_thresh,Round,Rating,RatingOnset,RT,Round_t1_idx,Round_t2_idx,Round_t3_idx,...,rpe_t3,cf_t1,cf_t2,cf_t3,cpe_t1,cpe_t2,cpe_t3,mood_epoch_len,next_round_start,norm_mood
0,MS002,14,low,4,70.0,539.343131,5.210698,3,2,1,...,-0.017374,0.551367,-0.307083,0.759848,1.461373,-0.892581,-0.795008,5.310698,545.592613,0.277664
1,MS002,14,low,7,70.0,572.673634,3.332630,6,5,4,...,-0.017374,-1.288170,0.674003,-0.920262,1.241834,0.802754,0.875934,3.432630,577.049490,0.277664
2,MS002,14,low,10,69.0,605.109393,3.499831,9,8,7,...,-1.720166,0.060824,-0.307083,-0.674991,-1.441430,0.266101,-1.185300,3.599831,609.642231,0.134538
3,MS002,14,low,13,71.0,638.791836,3.884104,12,11,10,...,-1.103047,1.164546,0.306096,0.428731,-1.197497,0.680788,-0.465698,3.984104,643.713750,0.420789
4,MS002,14,low,16,71.0,671.977904,4.250371,15,14,13,...,-0.017374,0.060824,0.796639,0.060824,1.729700,-0.831598,-0.831598,4.350371,677.264040,0.420789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,DA039,22,high,139,45.0,2176.828649,2.673696,138,137,136,...,-0.887088,NaN,-0.479399,0.915739,NaN,0.314126,-0.772921,2.773696,2180.513505,-0.524890
1246,DA039,22,high,142,45.0,2211.211948,2.961470,141,140,139,...,0.046736,0.760724,-3.362685,-0.014353,-0.652138,1.835992,-0.289789,3.061470,2215.197736,-0.524890
1247,DA039,22,high,145,55.0,2246.799223,3.347751,144,143,142,...,NaN,-1.967547,0.760724,NaN,1.111294,1.884306,NaN,3.447751,2251.180979,0.330135
1248,DA039,22,high,148,68.0,2276.912826,4.403968,147,146,145,...,-1.152672,-2.339584,0.605708,0.760724,1.763523,1.497800,-0.652138,4.503968,2282.349445,1.441667


# Feature Selection for Mood Models

In [33]:
mood_model_data = all_mood.copy()
mood_model_data = mood_model_data[mood_model_data.keep_mood=='keep']
mood_model_data

,subj_id,bdi,bdi_thresh,Round,Rating,RatingOnset,RT,Round_t1_idx,Round_t2_idx,Round_t3_idx,...,rpe_t3,cf_t1,cf_t2,cf_t3,cpe_t1,cpe_t2,cpe_t3,mood_epoch_len,next_round_start,norm_mood
0,MS002,14,low,4,70.0,539.343131,5.210698,3,2,1,...,-0.017374,0.551367,-0.307083,0.759848,1.461373,-0.892581,-0.795008,5.310698,545.592613,0.277664
1,MS002,14,low,7,70.0,572.673634,3.332630,6,5,4,...,-0.017374,-1.288170,0.674003,-0.920262,1.241834,0.802754,0.875934,3.432630,577.049490,0.277664
2,MS002,14,low,10,69.0,605.109393,3.499831,9,8,7,...,-1.720166,0.060824,-0.307083,-0.674991,-1.441430,0.266101,-1.185300,3.599831,609.642231,0.134538
3,MS002,14,low,13,71.0,638.791836,3.884104,12,11,10,...,-1.103047,1.164546,0.306096,0.428731,-1.197497,0.680788,-0.465698,3.984104,643.713750,0.420789
4,MS002,14,low,16,71.0,671.977904,4.250371,15,14,13,...,-0.017374,0.060824,0.796639,0.060824,1.729700,-0.831598,-0.831598,4.350371,677.264040,0.420789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,DA039,22,high,139,45.0,2176.828649,2.673696,138,137,136,...,-0.887088,NaN,-0.479399,0.915739,NaN,0.314126,-0.772921,2.773696,2180.513505,-0.524890
1246,DA039,22,high,142,45.0,2211.211948,2.961470,141,140,139,...,0.046736,0.760724,-3.362685,-0.014353,-0.652138,1.835992,-0.289789,3.061470,2215.197736,-0.524890
1247,DA039,22,high,145,55.0,2246.799223,3.347751,144,143,142,...,NaN,-1.967547,0.760724,NaN,1.111294,1.884306,NaN,3.447751,2251.180979,0.330135
1248,DA039,22,high,148,68.0,2276.912826,4.403968,147,146,145,...,-1.152672,-2.339584,0.605708,0.760724,1.763523,1.497800,-0.652138,4.503968,2282.349445,1.441667


In [34]:
mood_model_data = mood_model_data[~mood_model_data.norm_mood.isnull()].reset_index(drop=True)
mood_model_data

,subj_id,bdi,bdi_thresh,Round,Rating,RatingOnset,RT,Round_t1_idx,Round_t2_idx,Round_t3_idx,...,rpe_t3,cf_t1,cf_t2,cf_t3,cpe_t1,cpe_t2,cpe_t3,mood_epoch_len,next_round_start,norm_mood
0,MS002,14,low,4,70.0,539.343131,5.210698,3,2,1,...,-0.017374,0.551367,-0.307083,0.759848,1.461373,-0.892581,-0.795008,5.310698,545.592613,0.277664
1,MS002,14,low,7,70.0,572.673634,3.332630,6,5,4,...,-0.017374,-1.288170,0.674003,-0.920262,1.241834,0.802754,0.875934,3.432630,577.049490,0.277664
2,MS002,14,low,10,69.0,605.109393,3.499831,9,8,7,...,-1.720166,0.060824,-0.307083,-0.674991,-1.441430,0.266101,-1.185300,3.599831,609.642231,0.134538
3,MS002,14,low,13,71.0,638.791836,3.884104,12,11,10,...,-1.103047,1.164546,0.306096,0.428731,-1.197497,0.680788,-0.465698,3.984104,643.713750,0.420789
4,MS002,14,low,16,71.0,671.977904,4.250371,15,14,13,...,-0.017374,0.060824,0.796639,0.060824,1.729700,-0.831598,-0.831598,4.350371,677.264040,0.420789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,DA039,22,high,139,45.0,2176.828649,2.673696,138,137,136,...,-0.887088,NaN,-0.479399,0.915739,NaN,0.314126,-0.772921,2.773696,2180.513505,-0.524890
1205,DA039,22,high,142,45.0,2211.211948,2.961470,141,140,139,...,0.046736,0.760724,-3.362685,-0.014353,-0.652138,1.835992,-0.289789,3.061470,2215.197736,-0.524890
1206,DA039,22,high,145,55.0,2246.799223,3.347751,144,143,142,...,NaN,-1.967547,0.760724,NaN,1.111294,1.884306,NaN,3.447751,2251.180979,0.330135
1207,DA039,22,high,148,68.0,2276.912826,4.403968,147,146,145,...,-1.152672,-2.339584,0.605708,0.760724,1.763523,1.497800,-0.652138,4.503968,2282.349445,1.441667


In [12]:
# mood_model_data = mood_model_data[~mood_model_data.norm_mood.isnull()].reset_index(drop=True)

mood_model_data = mood_model_data[~mood_model_data.cpe_t1.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.cpe_t2.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.cpe_t3.isnull()].reset_index(drop=True)

# mood_model_data = mood_model_data[~mood_model_data.CR_t1.isnull()].reset_index(drop=True)
# mood_model_data = mood_model_data[~mood_model_data.CR_t2.isnull()].reset_index(drop=True)
# mood_model_data = mood_model_data[~mood_model_data.CR_t3.isnull()].reset_index(drop=True)

# mood_model_data = mood_model_data[~mood_model_data.choiceEV_t1.isnull()].reset_index(drop=True)
# mood_model_data = mood_model_data[~mood_model_data.choiceEV_t2.isnull()].reset_index(drop=True)
# mood_model_data = mood_model_data[~mood_model_data.choiceEV_t3.isnull()].reset_index(drop=True)

mood_model_data = mood_model_data[~mood_model_data.GambleEV_t1.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.GambleEV_t2.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.GambleEV_t3.isnull()].reset_index(drop=True)

mood_model_data = mood_model_data[~mood_model_data.TrialEV_t1.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.TrialEV_t2.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.TrialEV_t3.isnull()].reset_index(drop=True)

mood_model_data = mood_model_data[~mood_model_data.TotalProfit_t1.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.TotalProfit_t2.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.TotalProfit_t3.isnull()].reset_index(drop=True)

mood_model_data = mood_model_data[~mood_model_data.Profit_t1.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.Profit_t2.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.Profit_t3.isnull()].reset_index(drop=True)

mood_model_data = mood_model_data[~mood_model_data.rpe_t1.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.rpe_t2.isnull()].reset_index(drop=True)
mood_model_data = mood_model_data[~mood_model_data.rpe_t3.isnull()].reset_index(drop=True)
mood_model_data


,subj_id,bdi,bdi_thresh,Round,Rating,RatingOnset,RT,Round_t1_idx,Round_t2_idx,Round_t3_idx,...,rpe_t3,cf_t1,cf_t2,cf_t3,cpe_t1,cpe_t2,cpe_t3,mood_epoch_len,next_round_start,norm_mood
0,MS002,14,low,4,70.0,539.343131,5.210698,3,2,1,...,-0.017374,0.551367,-0.307083,0.759848,1.461373,-0.892581,-0.795008,5.310698,545.592613,0.277664
1,MS002,14,low,7,70.0,572.673634,3.332630,6,5,4,...,-0.017374,-1.288170,0.674003,-0.920262,1.241834,0.802754,0.875934,3.432630,577.049490,0.277664
2,MS002,14,low,10,69.0,605.109393,3.499831,9,8,7,...,-1.720166,0.060824,-0.307083,-0.674991,-1.441430,0.266101,-1.185300,3.599831,609.642231,0.134538
3,MS002,14,low,13,71.0,638.791836,3.884104,12,11,10,...,-1.103047,1.164546,0.306096,0.428731,-1.197497,0.680788,-0.465698,3.984104,643.713750,0.420789
4,MS002,14,low,16,71.0,671.977904,4.250371,15,14,13,...,-0.017374,0.060824,0.796639,0.060824,1.729700,-0.831598,-0.831598,4.350371,677.264040,0.420789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,DA039,22,high,130,62.0,2068.532096,3.829234,129,128,127,...,0.046736,0.915739,0.605708,-0.882439,0.676475,-0.531355,0.386596,3.929234,2073.394432,0.928652
1059,DA039,22,high,136,45.0,2141.050732,2.415390,135,134,133,...,0.046736,-0.014353,-0.014353,-0.014353,-1.014487,-0.289789,-0.772921,2.515390,2144.478929,-0.524890
1060,DA039,22,high,142,45.0,2211.211948,2.961470,141,140,139,...,0.046736,0.760724,-3.362685,-0.014353,-0.652138,1.835992,-0.289789,3.061470,2215.197736,-0.524890
1061,DA039,22,high,148,68.0,2276.912826,4.403968,147,146,145,...,-1.152672,-2.339584,0.605708,0.760724,1.763523,1.497800,-0.652138,4.503968,2282.349445,1.441667


In [13]:
mood_model_data.to_csv(f'{paper_dir}data/mood/mood_model_data_{date}.csv')
mood_model_data.to_csv(f'{paper_dir}data/mood/mood_model_data.csv')

In [18]:
mood_model_data.columns

Index(['subj_id', 'bdi', 'bdi_thresh', 'Round', 'Rating', 'RatingOnset', 'RT',
       'Round_t1_idx', 'Round_t2_idx', 'Round_t3_idx', 'epoch', 'epoch_t1_idx',
       'epoch_t2_idx', 'epoch_t3_idx', 'keep_mood', 'logRT', 'MoodChoiceOnset',
       'SafeBet_t1', 'SafeBet_t2', 'SafeBet_t3', 'LowBet_t1', 'LowBet_t2',
       'LowBet_t3', 'HighBet_t1', 'HighBet_t2', 'HighBet_t3', 'Profit_t1',
       'Profit_t2', 'Profit_t3', 'TotalProfit_t1', 'TotalProfit_t2',
       'TotalProfit_t3', 'GambleEV_t1', 'GambleEV_t2', 'GambleEV_t3',
       'TrialEV_t1', 'TrialEV_t2', 'TrialEV_t3', 'CR_t1', 'CR_t2', 'CR_t3',
       'choiceEV_t1', 'choiceEV_t2', 'choiceEV_t3', 'rpe_t1', 'rpe_t2',
       'rpe_t3', 'cf_t1', 'cf_t2', 'cf_t3', 'cpe_t1', 'cpe_t2', 'cpe_t3',
       'mood_epoch_len', 'next_round_start', 'norm_mood'],
      dtype='object')

# Create Variable Combos for Models

In [14]:
mood_model_data.columns

Index(['subj_id', 'bdi', 'bdi_thresh', 'Round', 'Rating', 'RatingOnset', 'RT',
       'Round_t1_idx', 'Round_t2_idx', 'Round_t3_idx', 'epoch', 'epoch_t1_idx',
       'epoch_t2_idx', 'epoch_t3_idx', 'keep_mood', 'logRT', 'MoodChoiceOnset',
       'SafeBet_t1', 'SafeBet_t2', 'SafeBet_t3', 'LowBet_t1', 'LowBet_t2',
       'LowBet_t3', 'HighBet_t1', 'HighBet_t2', 'HighBet_t3', 'Profit_t1',
       'Profit_t2', 'Profit_t3', 'TotalProfit_t1', 'TotalProfit_t2',
       'TotalProfit_t3', 'GambleEV_t1', 'GambleEV_t2', 'GambleEV_t3',
       'TrialEV_t1', 'TrialEV_t2', 'TrialEV_t3', 'CR_t1', 'CR_t2', 'CR_t3',
       'choiceEV_t1', 'choiceEV_t2', 'choiceEV_t3', 'rpe_t1', 'rpe_t2',
       'rpe_t3', 'cf_t1', 'cf_t2', 'cf_t3', 'cpe_t1', 'cpe_t2', 'cpe_t3',
       'mood_epoch_len', 'next_round_start', 'norm_mood'],
      dtype='object')

In [17]:
main_var = 'cpe'
iter_vars = ['GambleEV','TrialEV','TotalProfit','rpe','Profit']
# iter_vars = ['SafeBet','LowBet','HighBet','GambleEV','TrialEV','TotalProfit']

var_combos = [[v,main_var] for v in iter_vars]

for i in range(1,len(iter_vars)-1):
    iter_list = [list(tup)+[main_var] for tup in list(itertools.combinations(iter_vars,i+1))]
    var_list  = []
    for iter in iter_list:
        if 'GambleEV' in iter:
            if 'TrialEV' in iter:
                continue
            elif 'CR' in iter:
                continue
            elif 'choiceEV' in iter:
                continue
            else:
                var_list.append(iter)

        elif 'TrialEV' in iter:
            if 'GambleEV' in iter:
                continue
            elif 'CR' in iter:
                continue
            elif 'choiceEV' in iter:
                continue
            else:
                var_list.append(iter)

        elif 'CR' in iter:
            if 'GambleEV' in iter:
                continue
            elif 'TrialEV' in iter:
                continue
            else:
                var_list.append(iter)

        elif 'choiceEV' in iter:
            if 'GambleEV' in iter:
                continue
            elif 'TrialEV' in iter:
                continue
            else:
                var_list.append(iter)        
        else:
            var_list.append(iter)
    var_combos.extend(var_list)


var_combos

[['GambleEV', 'cpe'],
 ['TrialEV', 'cpe'],
 ['TotalProfit', 'cpe'],
 ['rpe', 'cpe'],
 ['Profit', 'cpe'],
 ['GambleEV', 'TotalProfit', 'cpe'],
 ['GambleEV', 'rpe', 'cpe'],
 ['GambleEV', 'Profit', 'cpe'],
 ['TrialEV', 'TotalProfit', 'cpe'],
 ['TrialEV', 'rpe', 'cpe'],
 ['TrialEV', 'Profit', 'cpe'],
 ['TotalProfit', 'rpe', 'cpe'],
 ['TotalProfit', 'Profit', 'cpe'],
 ['rpe', 'Profit', 'cpe'],
 ['GambleEV', 'TotalProfit', 'rpe', 'cpe'],
 ['GambleEV', 'TotalProfit', 'Profit', 'cpe'],
 ['GambleEV', 'rpe', 'Profit', 'cpe'],
 ['TrialEV', 'TotalProfit', 'rpe', 'cpe'],
 ['TrialEV', 'TotalProfit', 'Profit', 'cpe'],
 ['TrialEV', 'rpe', 'Profit', 'cpe'],
 ['TotalProfit', 'rpe', 'Profit', 'cpe'],
 ['GambleEV', 'TotalProfit', 'rpe', 'Profit', 'cpe'],
 ['TrialEV', 'TotalProfit', 'rpe', 'Profit', 'cpe']]

In [19]:
len(var_combos)

23

# Mood Models
- run cpe models + save outputs 

In [20]:
model_list = ['_'.join(vars) for vars in var_combos]
model_list

['GambleEV_cpe',
 'TrialEV_cpe',
 'TotalProfit_cpe',
 'rpe_cpe',
 'Profit_cpe',
 'GambleEV_TotalProfit_cpe',
 'GambleEV_rpe_cpe',
 'GambleEV_Profit_cpe',
 'TrialEV_TotalProfit_cpe',
 'TrialEV_rpe_cpe',
 'TrialEV_Profit_cpe',
 'TotalProfit_rpe_cpe',
 'TotalProfit_Profit_cpe',
 'rpe_Profit_cpe',
 'GambleEV_TotalProfit_rpe_cpe',
 'GambleEV_TotalProfit_Profit_cpe',
 'GambleEV_rpe_Profit_cpe',
 'TrialEV_TotalProfit_rpe_cpe',
 'TrialEV_TotalProfit_Profit_cpe',
 'TrialEV_rpe_Profit_cpe',
 'TotalProfit_rpe_Profit_cpe',
 'GambleEV_TotalProfit_rpe_Profit_cpe',
 'TrialEV_TotalProfit_rpe_Profit_cpe']

In [21]:
model_inputs = {}

for vars in var_combos:
    model_id = '_'.join(vars)
    model_data_vars = ['norm_mood']
    for v in vars:
        model_data_vars.extend(['_'.join([v,'t1']),'_'.join([v,'t2']),'_'.join([v,'t3'])])
    model_inputs[model_id] = model_data_vars


In [ ]:
mood_glm_pyEM

In [22]:
model_results = {f'{model}':{f'{subj_id}':{} for subj_id in subj_ids} for model in model_inputs.keys()}
gamma_bounds  = [(0.00001,1)]
n_inits = 3

for model_id in list(model_inputs.keys()):
    n_regs       = len(model_id.split('_'))
    param_bounds = gamma_bounds + [(-2,2) for reg_num in range(n_regs+1)]
    gam_inits    = np.array([np.random.uniform(0,1) for n in range(n_inits)])
    var_inits    = [np.random.uniform(-1,1,n_inits) for n in range(n_regs+1)]

    for subj_id in subj_ids:
        subj_df = mood_model_data[mood_model_data.subj_id == subj_id][model_inputs[model_id]]
        var_init_grid  = itertools.product(*np.vstack((gam_inits,var_inits))) 
        subj_model_fit = parallel_glm_fit(min_negll_glm,fit_swb_glm,var_init_grid,param_bounds,subj_df)
        model_results[model_id][subj_id] = subj_model_fit

pickle.dump(model_results,open(f'{paper_dir}data/mood/mood_model_results_dict_{date}.pkl','wb'))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  66 out of  81 | elapsed:    3.7s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    3.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  66 out of  81 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n

In [23]:
f'{paper_dir}data/mood/mood_model_results_dict_{date}.pkl'

'/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/manuscript/data/mood/mood_model_results_dict_11212024.pkl'

In [ ]:
#variable transformation functions
def norm2riskaversion(aversion_param):
    return 2 / (1 + np.exp(-aversion_param))
def norm2lossaversion(aversion_param):
    return 6 / (1 + np.exp(-aversion_param))
def norm2invtmp(invtemp):
    return 10 / (1 + np.exp(-invtemp))

#negll calculation and fit fn (update MLE fns to have two output options!)
def swb_pt_pyEM(params, subj_df, prior=None, output='npl'):

    risk_aversion, loss_aversion, inverse_temp = params
    
    risk_aversion = norm2riskaversion(risk_aversion) #transform parameter from gaussian space back into native model space using parameter-specific sigmoid function
    risk_aversion_bounds = [0.00001, 2] #set upper and lower bounds
    if risk_aversion< min(risk_aversion_bounds) or risk_aversion> max(risk_aversion_bounds): #prevent estimation from parameter values outside of bounds 
        return 10000000
    
    loss_aversion = norm2lossaversion(loss_aversion) #transform parameter from gaussian space back into native model space using parameter-specific sigmoid function
    loss_aversion_bounds = [0.00001, 6] #set upper and lower bounds
    if loss_aversion< min(loss_aversion_bounds) or loss_aversion> max(loss_aversion_bounds):  #prevent estimation from parameter values outside of bounds 
        return 10000000
    
    inverse_temp = norm2invtmp(inverse_temp) #transform parameter from gaussian space back into native model space using parameter-specific sigmoid function
    this_beta_bounds = [0.00001, 10]  #set upper and lower bounds
    if inverse_temp < min(this_beta_bounds) or inverse_temp > max(this_beta_bounds):  #prevent estimation from parameter values outside of bounds 
        return 10000000


    subj_df['util_HighBet'] = 0.5 * ((subj_df.HighBet)**risk_aversion)
    subj_df['util_LowBet']  = -0.5 * loss_aversion * ((-subj_df.LowBet)**risk_aversion)
    subj_df['util_SafeBet'] = [(safe)**risk_aversion if safe >= 0 else -loss_aversion*((-safe)**risk_aversion) for safe in subj_df.SafeBet.tolist()]
    subj_df['util_Gamble']  = subj_df['util_HighBet'] + subj_df['util_LowBet']
    subj_df['P_Gamble']     = np.exp(inverse_temp*subj_df['util_Gamble']) / ( np.exp(inverse_temp*subj_df['util_Gamble']) + np.exp(inverse_temp*subj_df['util_SafeBet']) )
    subj_df['P_Safe']       = np.exp(inverse_temp*subj_df['util_SafeBet']) / ( np.exp(inverse_temp*subj_df['util_Gamble']) + np.exp(inverse_temp*subj_df['util_SafeBet']) )

    choice_prob_key = {'gamble':'P_Gamble','safe':'P_Safe'}
    subj_df['Choice_Prob']  = [row[choice_prob_key[row['GambleChoice']]] if (row['GambleChoice']=='gamble') | (row['GambleChoice']=='safe') else np.nan for ix,row in subj_df.iterrows()]

    choice_util_key = {'gamble':'util_Gamble','safe':'util_SafeBet'}
    subj_df['Choice_Util']  = [row[choice_util_key[row['GambleChoice']]] if (row['GambleChoice']=='gamble') | (row['GambleChoice']=='safe') else np.nan for ix,row in subj_df.iterrows()]

    choice_prob_list = subj_df.Choice_Prob.dropna()

    # calculate negative log likelihood of choice probabilities 
    negll = -np.sum(np.log(choice_prob_list))
    
    if np.isnan(negll):
        negll = np.inf
    
    if output == 'npl':
        if prior is not None:  # EM-fit: P(Choices | h) * P(h | O) should be maximised, therefore same as minimizing it with negative sign
            fval = -(-negll + prior['logpdf'](params))

            if any(prior['sigma'] == 0):
                this_mu = prior['mu']
                this_sigma = prior['sigma']
                this_logprior = prior['logpdf'](params)
                print(f'mu: {this_mu}')
                print(f'sigma: {this_sigma}')
                print(f'logpdf: {this_logprior}')
                print(f'fval: {fval}')
            
            if np.isinf(fval): 
                fval = 10000000
            return fval
        else: # NLL fit 
            return negll
        
    elif output == 'all': #WHATEVER YOU WANT TO OUTPUT IF NOT OPTIMIZING
        res_dict = {'params' : [risk_aversion, loss_aversion, inverse_temp],
                     'negll' : negll,
                     'bic'   : len(params) * np.log(len(choice_prob_list)) + 2*negll} #K*np.log(len(pred_mood)) + 2*negll}
        
        return subj_df,res_dict